# Julia methods as actors

## Installing necessary requirements

1) Install Julia itself. Start from here: <http://julialang.org/downloads/>

2) Install `PyCall` which is a Python binding library for Julia (i.e. calling Python from Julia).

```julia
julia> Pkg.update()
julia> Pkg.add("PyCall")
```

3) Install `pyjulia` Python package:

3a) Download it from github.com: 

```bash
git clone git@github.com:JuliaLang/pyjulia.git
```

3b) Install the package:

```bash
cd pyjulia
pip install .              # Copy to site-packages
# pip install -e .         # Makes link to current directory in site-packages
```

4) Try in Python:

```python
import julia
jl = julia.Julia()        # Takes a few seconds
jl.eval("2 + 2")          # Should immediately return "4"
```

In [1]:
%load_ext autoreload
%autoreload(2)

In [2]:
from wowp.actors.julia import JuliaMethod
from wowp.schedulers import LinearizedScheduler
import numpy as np

## Simple calling

In [3]:
sqrt = JuliaMethod("sqrt", inports="a")
sqrt(4)

2.0

## Calling on numpy arrays

In [4]:
sqrt = JuliaMethod("sqrt", inports="a")

array = np.random.rand(5, 5)
scheduler = LinearizedScheduler()
scheduler.put_value(sqrt.inports.a, array)
scheduler.execute()
sqrt.outports.result.pop()

array([[ 0.66115551,  0.93729044,  0.94669025,  0.70866149,  0.54879759],
       [ 0.56510828,  0.73486272,  0.60286925,  0.4115851 ,  0.71968276],
       [ 0.42043387,  0.43278543,  0.58673638,  0.59730121,  0.82560461],
       [ 0.86499419,  0.16210683,  0.72426582,  0.38206654,  0.82500869],
       [ 0.30027383,  0.25691385,  0.76714407,  0.7052992 ,  0.72146483]])

## Chain sqrt method to pass numpy arrays

In [5]:
sqrt = JuliaMethod("sqrt", inports="a")
sqrt2 = JuliaMethod("sqrt", inports="a")

sqrt.outports.result.connect(sqrt2.inports.a)

array = np.random.rand(5, 5)
scheduler = LinearizedScheduler()
scheduler.put_value(sqrt.inports.a, array)
scheduler.execute()
sqrt2.outports.result.pop()

array([[ 0.76990142,  0.80135374,  0.42029207,  0.82993179,  0.99130717],
       [ 0.89247752,  0.50644769,  0.75747862,  0.94370161,  0.92042877],
       [ 0.85841894,  0.52022746,  0.90292523,  0.93843234,  0.91497422],
       [ 0.84477496,  0.89693243,  0.4885782 ,  0.80501227,  0.96402427],
       [ 0.73452196,  0.48977517,  0.95385857,  0.59077172,  0.9035825 ]])

## Using method from a package

In [6]:
%%file ABCD.jl

module ABCD

VERSION < v"0.4-" && using Docile

export quad

@doc doc"""Fourth power of the argument.""" ->
function quad(a)
    a ^ 4
end

end

Overwriting ABCD.jl


In [7]:
quad = JuliaMethod(package_name="ABCD", method_name="quad", inports="a")
quad(4.0)

256.0

In [8]:
quad.name

'ABCD.quad'

### Non-existent module

In [9]:
xxx = JuliaMethod(package_name="ABBD", method_name="x")
xxx()

JuliaError: Exception 'ErrorException' ocurred while calling julia code:
@VD

Code:
using ABBD

In [10]:
xxx = JuliaMethod(package_name="ABCD", method_name="xx")
xxx()

JuliaError: Exception 'UndefVarError' ocurred while calling julia code:
@VD

Code:
ABCD.xx